<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-26 11:10:18
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.38 C
-------------------
Today PnL: 63.36 K (0.46%)
Current PnL: -36.02 L (-22.51%)
CY Booked + Current PnL: -20.34 L (-12.71%)
-------------------
Total profit:  1.50 L
Total loss:  -37.52 L
-------------------
Total Booked + Current PnL: 6.64 L (5.0%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.33%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.03 C (74.14%)
Deployed:  1.33 C
Current:  1.38 C
CAGR/XIRR %: 2.04%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.01,6.76,10.22,17.67,16417.0,10176.0,160636.0,478.51,29.12,59.0,L-LC,11.11,185.0,0.62,1.18,28.95,XY25,ATH,MINING
50,MASFIN,0.01,4.10,16.78,21.57,17116.0,4020.0,102000.0,397.04,-10.12,59.0,H-SC,6.58,167.0,0.23,0.75,48.85,XR,ATH,FINANCE
77,TTKPRESTIG,-0.60,-31.39,45.89,0.09,31729.0,-31635.0,69142.0,770.00,60.51,32.0,M-SC,11.21,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
37,ICICIGI,0.29,3.03,15.83,19.34,32637.0,6061.0,206170.0,2252.93,-13.21,57.0,X-MC,1.19,66.0,0.19,1.52,19.45,X40,ATH,INSURANCE
51,MEDANTA,0.06,-3.50,28.58,24.08,35012.0,-4446.0,122504.0,1486.00,-4.25,53.0,X-SC,8.35,87.0,-0.13,0.90,12.20,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,7.80,-11.18,63.90,45.57,111878.0,-22049.0,175083.0,844.00,3797.30,63.0,M-SC,6.01,248.0,-0.20,1.29,32.50,XY24,NTT,AUTO
71,TANLA,5.71,-47.14,315.59,119.69,451710.0,-127629.0,143132.0,1943.92,-46.61,47.0,H-SC,19.86,147.0,-0.28,1.05,13.07,AR,ATH,IT
26,EASEMYTRIP,5.64,-50.39,193.66,45.70,169886.0,-89091.0,87724.0,26.40,-8.83,65.0,M-SC,18.34,204.0,-0.52,0.65,44.53,XY24,NTT,TRAVEL
27,FINCABLES,3.56,8.42,91.77,107.91,146898.0,12428.0,160072.0,1641.55,-2.59,67.0,M-SC,7.47,220.0,0.08,1.18,20.43,OX40N,ATH,CABLES
15,BSOFT,3.56,-30.20,116.03,50.79,113912.0,-42478.0,98175.0,831.70,-10.32,43.0,H-SC,5.46,172.0,-0.37,0.72,14.31,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-1.89,1.80,18.63,20.77,44406.0,4214.0,238358.0,1644.00,-9.71,49.0,X-MC,1.11,64.0,0.09,1.75,9.33,X40N,NTT,BREWERIES
5,ANGELONE,-1.72,-89.98,1139.48,24.20,348544.0,-274662.0,30588.0,3033.00,-85.37,43.0,X-SC,8.50,97.0,-0.79,0.22,58.10,X40N,NTT,FINANCE
52,PAGEIND,-1.10,-18.08,58.87,30.14,114320.0,-42864.0,194190.0,51418.64,-39.45,36.0,X-MC,18.44,55.0,-0.37,1.43,0.00,X40,ATH,APPARELS
55,RAJESHEXPO,-1.06,-70.90,243.91,0.07,97354.0,-97263.0,39914.0,518.00,1363.38,33.0,L-SC,9.48,268.0,-1.00,0.29,0.00,OX40N,NTT,JEWELLERY
76,TRIDENT,-1.04,-31.16,86.70,28.51,55058.0,-28750.0,63504.0,48.00,-11.55,44.0,M-SC,8.76,223.0,-0.52,0.47,8.66,XR,NTT,TEXTILES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,3.56,8.42,91.77,107.91,146898.0,12428.0,160072.0,1641.55,-2.59,67.0,M-SC,7.47,220.0,0.08,1.18,20.43,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.60,-31.39,45.89,0.09,31729.0,-31635.0,69142.0,770.00,60.51,32.0,M-SC,11.21,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
66,SIS,1.88,-31.41,76.80,21.26,58488.0,-34876.0,76156.0,528.00,1787.05,35.0,H-SC,5.22,168.0,-0.60,0.56,4.99,OX40N,NTT,MISC
41,INDIGOPNTS,0.08,-32.61,48.41,0.02,56949.0,-56920.0,117639.0,1408.00,79.13,28.0,M-SC,11.71,222.0,-1.00,0.87,2.33,OX40N,NTT,PAINTS
48,KANSAINER,0.23,-32.88,69.07,13.47,125010.0,-88677.0,180990.0,340.00,-42.28,34.0,H-SC,12.43,162.0,-0.71,1.33,1.74,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1.77,21.2,19.8,45.2,46014.0,40653.0,232395.0,1856.0,38.75,64.0,X-MC,8.41,79.0,0.88,1.71,29.44,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.01,6.76,10.22,17.67,16417.0,10176.0,160636.0,478.51,29.12,59.0,L-LC,11.11,185.0,0.62,1.18,28.95,XY25,ATH,MINING
27,FINCABLES,3.56,8.42,91.77,107.91,146898.0,12428.0,160072.0,1641.55,-2.59,67.0,M-SC,7.47,220.0,0.08,1.18,20.43,OX40N,ATH,CABLES
50,MASFIN,0.01,4.10,16.78,21.57,17116.0,4020.0,102000.0,397.04,-10.12,59.0,H-SC,6.58,167.0,0.23,0.75,48.85,XR,ATH,FINANCE
1,ABB,-0.11,13.04,28.78,45.57,85111.0,34109.0,295728.0,7934.00,-27.37,70.0,H-MC,8.70,125.0,0.40,2.17,31.32,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.11,13.04,28.78,45.57,85111.0,34109.0,295728.0,7934.00,-27.37,70.0,H-MC,8.70,125.0,0.40,2.17,31.32,AR,NTT,ELECTRICAL
50,MASFIN,0.01,4.10,16.78,21.57,17116.0,4020.0,102000.0,397.04,-10.12,59.0,H-SC,6.58,167.0,0.23,0.75,48.85,XR,ATH,FINANCE
27,FINCABLES,3.56,8.42,91.77,107.91,146898.0,12428.0,160072.0,1641.55,-2.59,67.0,M-SC,7.47,220.0,0.08,1.18,20.43,OX40N,ATH,CABLES
85,ZYDUSLIFE,1.70,-1.37,37.51,35.63,77534.0,-2873.0,206701.0,1286.17,-13.96,61.0,H-MC,4.90,120.0,-0.04,1.52,15.07,AR,ATH,PHARMA
40,INDIAMART,-0.48,-6.16,120.29,106.72,139225.0,-7595.0,115741.0,4810.62,-57.62,46.0,H-SC,8.01,140.0,-0.05,0.85,15.78,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-0.48,-38.48,155.52,57.19,312018.0,-125498.0,200629.0,485.00,-64.75,19.0,X-SC,18.65,81.0,-0.40,1.48,0.00,XY24,NTT,FMCG
43,INFY,0.64,-11.61,51.88,34.24,143479.0,-36329.0,276559.0,1972.00,-30.33,23.0,X-LC,7.95,3.0,-0.25,2.03,1.80,X40,NTT,IT
34,HCLTECH,0.43,-10.18,37.87,23.83,82293.0,-24620.0,217304.0,1908.19,-5.71,28.0,X-LC,5.38,8.0,-0.30,1.60,4.26,X40,ATH,IT
53,PGHH,-0.11,-14.35,54.13,32.01,98993.0,-30640.0,182880.0,17616.87,-42.03,30.0,X-MC,9.62,57.0,-0.31,1.34,0.00,X40,ATH,FMCG
12,BATAINDIA,0.77,-48.16,160.81,35.20,107265.0,-61967.0,66703.0,2096.00,-12.49,32.0,X-SC,22.11,91.0,-0.58,0.49,0.77,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-1.89,1.80,18.63,20.77,44406.0,4214.0,238358.0,1644.00,-9.71,49.0,X-MC,1.11,64.0,0.09,1.75,9.33,X40N,NTT,BREWERIES
37,ICICIGI,0.29,3.03,15.83,19.34,32637.0,6061.0,206170.0,2252.93,-13.21,57.0,X-MC,1.19,66.0,0.19,1.52,19.45,X40,ATH,INSURANCE
22,DABUR,0.25,1.66,41.36,43.70,104300.0,4108.0,252176.0,735.00,-7.83,56.0,X-MC,1.31,70.0,0.04,1.85,16.96,XY24,BTT,FMCG
59,RELIANCE,0.46,-9.81,38.93,25.30,70559.0,-19718.0,181245.0,1952.00,-12.34,42.0,X-LC,2.52,21.0,-0.28,1.33,21.41,XY25,BTT,REFINERIES
35,HINDUNILVR,-0.16,-7.77,23.24,13.67,55102.0,-19963.0,237100.0,2922.00,-31.53,52.0,X-LC,2.61,9.0,-0.36,1.74,11.93,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-0.48,-38.48,155.52,57.19,312018.0,-125498.0,200629.0,485.00,-64.75,19.0,X-SC,18.65,81.0,-0.40,1.48,0.00,XY24,NTT,FMCG
53,PGHH,-0.11,-14.35,54.13,32.01,98993.0,-30640.0,182880.0,17616.87,-42.03,30.0,X-MC,9.62,57.0,-0.31,1.34,0.00,X40,ATH,FMCG
52,PAGEIND,-1.10,-18.08,58.87,30.14,114320.0,-42864.0,194190.0,51418.64,-39.45,36.0,X-MC,18.44,55.0,-0.37,1.43,0.00,X40,ATH,APPARELS
2,ABBOTINDIA,-0.04,-11.44,33.31,18.07,52763.0,-20455.0,158400.0,35195.00,-24.65,39.0,X-MC,11.33,58.0,-0.39,1.16,0.42,X40,ATH,PHARMA
58,RELAXO,0.14,-54.09,231.78,52.31,154451.0,-78523.0,66637.0,1176.00,-51.12,38.0,X-SC,14.52,92.0,-0.51,0.49,0.42,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-1.72,-89.98,1139.48,24.20,348544.0,-274662.0,30588.0,3033.0,-85.37,43.0,X-SC,8.50,97.0,-0.79,0.22,58.10,X40N,NTT,FINANCE
54,QUESS,-0.21,-33.82,115.84,42.84,49834.0,-21986.0,43020.0,424.0,-56.47,39.0,X-SC,17.38,80.0,-0.44,0.32,3.48,XY24,NTT,MISC
58,RELAXO,0.14,-54.09,231.78,52.31,154451.0,-78523.0,66637.0,1176.0,-51.12,38.0,X-SC,14.52,92.0,-0.51,0.49,0.42,X40N,NTT,FOOTWEAR
12,BATAINDIA,0.77,-48.16,160.81,35.20,107265.0,-61967.0,66703.0,2096.0,-12.49,32.0,X-SC,22.11,91.0,-0.58,0.49,0.77,X40,NTT,FOOTWEAR
51,MEDANTA,0.06,-3.50,28.58,24.08,35012.0,-4446.0,122504.0,1486.0,-4.25,53.0,X-SC,8.35,87.0,-0.13,0.90,12.20,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,-0.22,-15.96,41.67,19.06,119122.0,-54297.0,285869.0,452.00,-53.90,43.0,X-LC,17.39,1.0,-0.46,2.10,3.51,X40,NTT,FMCG
73,TCS,0.86,-23.99,62.59,23.58,205810.0,-103806.0,328823.0,4311.58,-34.12,32.0,X-LC,8.47,2.0,-0.50,2.42,3.03,X40,ATH,IT
43,INFY,0.64,-11.61,51.88,34.24,143479.0,-36329.0,276559.0,1972.00,-30.33,23.0,X-LC,7.95,3.0,-0.25,2.03,1.80,X40,NTT,IT
75,TMPV,2.41,-13.10,53.43,33.33,151271.0,-42680.0,283120.0,600.00,-81.44,66.0,X-LC,3.66,4.0,-0.28,2.08,15.75,XY24,NTT,AUTO
34,HCLTECH,0.43,-10.18,37.87,23.83,82293.0,-24620.0,217304.0,1908.19,-5.71,28.0,X-LC,5.38,8.0,-0.30,1.60,4.26,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INDUSINDBK,1.35,-29.86,89.47,32.89,48448.0,-23056.0,54150.0,1800.00,-354.90,63.0,L-MC,11.77,258.0,-0.48,0.40,49.12,XR,NTT,BANKS
67,SONACOMS,-0.53,-6.73,49.13,39.09,46355.0,-6808.0,94351.0,804.02,-26.14,63.0,M-MC,10.59,193.0,-0.15,0.69,33.48,AR,ATH,AUTO
50,MASFIN,0.01,4.10,16.78,21.57,17116.0,4020.0,102000.0,397.04,-10.12,59.0,H-SC,6.58,167.0,0.23,0.75,48.85,XR,ATH,FINANCE
19,COALINDIA,-1.01,6.76,10.22,17.67,16417.0,10176.0,160636.0,478.51,29.12,59.0,L-LC,11.11,185.0,0.62,1.18,28.95,XY25,ATH,MINING
65,SIEMENS,0.38,-9.63,38.88,25.51,65388.0,-17915.0,168180.0,4671.50,41.28,65.0,H-LC,3.25,51.0,-0.27,1.24,23.94,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,5.64,-50.39,193.66,45.70,169886.0,-89091.0,87724.0,26.40,-8.83,65.0,M-SC,18.34,204.0,-0.52,0.65,44.53,XY24,NTT,TRAVEL
7,ATULAUTO,7.80,-11.18,63.90,45.57,111878.0,-22049.0,175083.0,844.00,3797.30,63.0,M-SC,6.01,248.0,-0.20,1.29,32.50,XY24,NTT,AUTO
1,ABB,-0.11,13.04,28.78,45.57,85111.0,34109.0,295728.0,7934.00,-27.37,70.0,H-MC,8.70,125.0,0.40,2.17,31.32,AR,NTT,ELECTRICAL
11,BANDHANBNK,2.95,-9.43,112.45,92.42,283706.0,-26264.0,252295.0,400.00,116.24,80.0,H-SC,7.71,174.0,-0.09,1.86,38.98,XY24,NTT,BANKS
27,FINCABLES,3.56,8.42,91.77,107.91,146898.0,12428.0,160072.0,1641.55,-2.59,67.0,M-SC,7.47,220.0,0.08,1.18,20.43,OX40N,ATH,CABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.75
1,25,45.84
2,50,77.84


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.98
MC    30.81
LC    25.18
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.65
X40      24.25
X40N     12.94
AR        9.33
XY25      9.06
XR        8.63
OX40N     7.27
SR        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.14
H-SC    22.68
X-LC    21.73
M-SC    10.99
X-SC     9.59
H-MC     5.15
M-MC     2.12
H-LC     1.24
L-LC     1.18
M-LC     1.03
L-SC     0.72
L-MC     0.40
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.90,-14.05,52.62
IT,11.50,-39.97,110.12
FINANCE,11.22,-36.71,90.51
MISC,6.97,-37.09,93.17
ELECTRICAL,6.52,-5.76,44.35
PAINTS,5.12,-32.68,52.40
INSURANCE,5.03,0.79,33.16
PHARMA,4.23,-2.15,34.30
AUTO,4.06,-12.95,56.01


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3368774.0,21
X40,1423660.0,15
X40N,1359265.0,10
XR,1318350.0,12
AR,1301781.0,10
OX40N,803311.0,10
XY25,383638.0,6
SR,304369.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3706486.0,24
X-SC,1500702.0,10
M-SC,1426779.0,14
X-MC,1363473.0,15
X-LC,1316776.0,12
H-MC,378069.0,3
L-SC,192105.0,2
M-LC,151719.0,1
M-MC,96786.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1245947.0      6
           AR         952129.0      5
           XR         843303.0      7
M-SC       XY24       826167.0      6
X-LC       X40        701519.0      6
X-SC       X40N       690985.0      4
           XY24       625358.0      4
X-MC       X40        537782.0      7
           X40N       441457.0      4
H-SC       OX40N      360738.0      4
M-SC       OX40N      345219.0      5
H-SC       SR         304369.0      2
X-LC       XY24       262773.0      2
           X40N       226823.0      2
H-MC       XY24       215424.0      1
X-MC       XY24       193105.0      2
           XY25       191129.0      2
X-SC       X40        184359.0      2
M-SC       XR         180129.0      2
H-MC       AR         162645.0      2
M-LC       XR         151719.0      1
X-LC       XY25       125661.0      2
L-SC       OX40N       97354.0      1
           XR          94751.0      1
M-SC       AR          75264.0      1
H-LC       AR          65388.0      1
M-MC       XY25        50431.0      1
L-MC       XR          48448.0      1
M-MC       AR          46355.0      1
L-LC       XY25        16417.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
